In [1]:
import os
import json
import pandas as pd
import torch
import ray.cloudpickle as pickle

In [2]:
main_folder_hyper = "HyperparameterLOG"

for folder_name in os.listdir(main_folder_hyper):
    folder_path = os.path.join(main_folder_hyper, folder_name)
    if os.path.isdir(folder_path):
        print(f"Subfolder name: {folder_name}")

Subfolder name: deeplabv3_resnet50
Subfolder name: deeplabv3_resnet101


In [3]:
# -----------------------------
# 1) DEFINE ABSOLUTE BASE PATH
# -----------------------------
BASE_PATH = "/home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation"
# Construct absolute path to your HyperparameterLOG
main_folder_hyper = os.path.join(BASE_PATH, "HyperparameterLOG")
main_folder_hyper = os.path.abspath(main_folder_hyper)
print("DEBUG: main_folder_hyper =", main_folder_hyper)
print("DEBUG: Exists on disk?   =", os.path.isdir(main_folder_hyper))

# (Optional) Print top-level subfolders for confirmation:
print("\nDEBUG: Subfolders in HyperparameterLOG:")
for item in os.listdir(main_folder_hyper):
    if os.path.isdir(os.path.join(main_folder_hyper, item)):
        print(" -", item)

# -----------------------------
# 2) LOAD HYPERPARAMETER RUNS
# -----------------------------
def load_hyperparameter_runs_as_dict(base_folder: str):
    runs_data = {}

    if not os.path.isdir(base_folder):
        print(f"ERROR: Base folder does not exist: {base_folder}")
        return runs_data

    for model_folder in os.listdir(base_folder):
        model_path = os.path.join(base_folder, model_folder)
        
        if not os.path.isdir(model_path):
            print(f"Skipping invalid model path: {model_path}")
            continue

        runs_data[model_folder] = {}

        for train_folder in os.listdir(model_path):
            train_folder_path = os.path.join(model_path, train_folder)

            # Sicherstellen, dass es sich um einen gültigen Trainingsordner handelt
            if not train_folder.startswith("train_hyper_") or not os.path.isdir(train_folder_path):
                print(f"Skipping non-training folder: {train_folder_path}")
                continue

            # Check if necessary files exist
            params_file = os.path.join(train_folder_path, "params.json")
            progress_file = os.path.join(train_folder_path, "progress.csv")
            result_file = os.path.join(train_folder_path, "result.json")

            if not (os.path.isfile(params_file) and os.path.isfile(progress_file) and os.path.isfile(result_file)):
                print(f"Skipping incomplete run: {train_folder_path}")
                continue

            # Laden der Dateien
            with open(params_file, "r", encoding="utf-8") as f:
                params_dict = json.load(f)

            progress_df = pd.read_csv(progress_file)
            progress_records = progress_df.to_dict(orient="records")

            result_records = []
            with open(result_file, "r", encoding="utf-8") as f:
                for line in f:
                    line = line.strip()
                    if line:
                        result_records.append(json.loads(line))

            run_dict = {
                "id": train_folder,
                **params_dict,
                "result": result_records,
                "progress": progress_records,
            }

            runs_data[model_folder][train_folder] = run_dict

    return runs_data

# Neuladen der Daten mit verbesserter Fehlerbehandlung
hyperparameter_data = load_hyperparameter_runs_as_dict(main_folder_hyper)




DEBUG: main_folder_hyper = /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/HyperparameterLOG
DEBUG: Exists on disk?   = True

DEBUG: Subfolders in HyperparameterLOG:
 - deeplabv3_resnet50
 - deeplabv3_resnet101
Skipping non-training folder: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/HyperparameterLOG/deeplabv3_resnet50/tuner.pkl
Skipping non-training folder: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/HyperparameterLOG/deeplabv3_resnet50/experiment_state-2025-02-06_19-30-16.json
Skipping incomplete run: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/HyperparameterLOG/deeplabv3_resnet50/train_hyper_ff4383cb_8_auto_cast=False,batch_size=4,learning_rate=0.0000,max_epochs=100,weight_decay=0.0038_2025-02-08_01-33-00
Skipping non-training folder: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/HyperparameterLOG/deeplabv3_resnet50/.validate_storage_marker
Skipping non-training folder:

In [4]:
# -----------------------------
# 3) SORT + FIND CHECKPOINTS
# -----------------------------
sorted_hyperparameter_data = {}

for model_name, runs_dict in hyperparameter_data.items():
    runs_list = []
    
    for run_name, run_data in runs_dict.items():
        # Determine best validation accuracy from the 'progress' CSV
        if "progress" in run_data and run_data["progress"]:
            best_val_acc = max(
                (r.get("val_acc", float("-inf")) for r in run_data["progress"]),
                default=float("-inf")
            )
        else:
            best_val_acc = float("-inf")
        
        # Build absolute path to this particular run folder
        run_folder_path = os.path.join(
            main_folder_hyper, model_name, run_name
        )
        run_folder_path = os.path.abspath(run_folder_path)

        # DEBUG: Print out the run folder path
        # print(f"DEBUG: run_folder_path for {run_name} = {run_folder_path}")
        
        # Gather checkpoint directories
        if os.path.isdir(run_folder_path):
            checkpoint_dirs = [
                d for d in os.listdir(run_folder_path)
                if d.startswith("checkpoint_") 
                   and os.path.isdir(os.path.join(run_folder_path, d))
            ]
        else:
            checkpoint_dirs = []
        
        # Debug printing
        print(f"\n[DEBUG] For model='{model_name}' run='{run_name}', found checkpoint dirs:")
        print("       ", checkpoint_dirs)

        # Pick the *last* checkpoint folder numerically
        if checkpoint_dirs:
            checkpoint_dirs.sort(key=lambda x: int(x.split("_")[1]))  # numeric sort by the suffix
            last_checkpoint_dir = checkpoint_dirs[-1]
            last_checkpoint = os.path.join(run_folder_path, last_checkpoint_dir, "checkpoint.pkl")
            last_checkpoint = os.path.abspath(last_checkpoint)
        else:
            last_checkpoint = None
        
        runs_list.append((run_name, run_data, best_val_acc, last_checkpoint))
    
    # Sort all runs by best_val_acc descending
    sorted_runs = sorted(runs_list, key=lambda x: x[2], reverse=True)
    
    # Build a new dictionary with simple int-string keys ("0", "1", ...)
    sorted_hyperparameter_data[model_name] = {
        str(i): {
            **run_data,
            "max_validation_accuracy": best_val_acc,
            "path": last_checkpoint
        }
        for i, (run_name, run_data, best_val_acc, last_checkpoint) in enumerate(sorted_runs)
    }



[DEBUG] For model='deeplabv3_resnet50' run='train_hyper_533faa74_5_auto_cast=False,batch_size=8,learning_rate=0.0006,max_epochs=100,weight_decay=0.0000_2025-02-08_01-32-26', found checkpoint dirs:
        []

[DEBUG] For model='deeplabv3_resnet50' run='train_hyper_1084e8d6_2_auto_cast=True,batch_size=8,learning_rate=0.0040,max_epochs=100,weight_decay=0.0076_2025-02-06_19-30-20', found checkpoint dirs:
        ['checkpoint_000002', 'checkpoint_000004']

[DEBUG] For model='deeplabv3_resnet50' run='train_hyper_ac0b31c1_6_auto_cast=False,batch_size=16,learning_rate=0.0027,max_epochs=100,weight_decay=0.0001_2025-02-08_01-32-38', found checkpoint dirs:
        []

[DEBUG] For model='deeplabv3_resnet50' run='train_hyper_8cdfeb36_4_auto_cast=False,batch_size=16,learning_rate=0.0000,max_epochs=100,weight_decay=0.0014_2025-02-08_01-32-14', found checkpoint dirs:
        []

[DEBUG] For model='deeplabv3_resnet50' run='train_hyper_e12c1511_1_auto_cast=True,batch_size=4,learning_rate=0.0001,max_ep

In [5]:

# -----------------------------
# 4) PRINT EXAMPLE CHECKPOINT PATH
# -----------------------------
# Example: print the best run's path for 'lraspp_mobilenet_v3_large'
# (Change "1" to "0" or "2" as needed, depending on how many runs you have.)
if "deeplabv3_resnet50" in sorted_hyperparameter_data:
    if "1" in sorted_hyperparameter_data["deeplabv3_resnet101"]:
        print("\nDEBUG: 'deeplabv3_resnet101' run #1 checkpoint path:")
        print(sorted_hyperparameter_data['deeplabv3_resnet101']['0']['path'])
    else:
        print("\nDEBUG: 'deeplabv3_resnet101' run #1 does not exist in dictionary.")

# You can print or debug other model-run combos similarly:
# print(sorted_hyperparameter_data["deeplabv3_resnet50"]["0"]["path"])

print(sorted_hyperparameter_data["deeplabv3_resnet50"]['0'].keys())
print(sorted_hyperparameter_data["deeplabv3_resnet50"]['0']['path'])

print(sorted_hyperparameter_data.keys())



DEBUG: 'deeplabv3_resnet101' run #1 checkpoint path:
/home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/HyperparameterLOG/deeplabv3_resnet101/train_hyper_1e767146_1_auto_cast=True,batch_size=4,learning_rate=0.0001,max_epochs=100,weight_decay=0.0000_2025-02-08_07-36-18/checkpoint_000099/checkpoint.pkl
dict_keys(['id', 'auto_cast', 'batch_size', 'learning_rate', 'max_epochs', 'weight_decay', 'result', 'progress', 'max_validation_accuracy', 'path'])
/home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/HyperparameterLOG/deeplabv3_resnet50/train_hyper_e12c1511_1_auto_cast=True,batch_size=4,learning_rate=0.0001,max_epochs=100,weight_decay=0.0000_2025-02-06_19-30-16/checkpoint_000099/checkpoint.pkl
dict_keys(['deeplabv3_resnet50', 'deeplabv3_resnet101'])


In [6]:
print(sorted_hyperparameter_data["deeplabv3_resnet50"]["0"].keys())

dict_keys(['id', 'auto_cast', 'batch_size', 'learning_rate', 'max_epochs', 'weight_decay', 'result', 'progress', 'max_validation_accuracy', 'path'])


In [7]:
# -----------------------------
# 5) REMAINDER OF YOUR EVAL CODE
# -----------------------------
import pandas as pd
from Helper.ml_models import TrainedModel, K_Fold_Dataset

def compute_confusion_matrix(predicted, ground_truth, num_classes):
    mask = (ground_truth >= 0) & (ground_truth < num_classes)
    label = num_classes * ground_truth[mask] + predicted[mask]
    count = torch.bincount(label, minlength=num_classes**2)
    confusion_matrix = count.reshape(num_classes, num_classes)
    return confusion_matrix

def compute_miou(confusion_matrix):
    intersection = torch.diag(confusion_matrix)
    ground_truth_set = confusion_matrix.sum(1)
    predicted_set = confusion_matrix.sum(0)
    union = ground_truth_set + predicted_set - intersection
    IoU = intersection / (union + 1e-6)
    mIoU = torch.mean(IoU)
    return mIoU.item(), IoU

def compute_mean_pixel_accuracy(confusion_matrix):
    true_positive = torch.diag(confusion_matrix)
    total_pixels = confusion_matrix.sum(1)
    pixel_accuracy = true_positive / (total_pixels + 1e-6)
    mPA = torch.mean(pixel_accuracy)
    return mPA.item(), pixel_accuracy

def compute_fwiou(confusion_matrix):
    total_pixels = confusion_matrix.sum()
    ground_truth_set = confusion_matrix.sum(1)
    intersection = torch.diag(confusion_matrix)
    union = ground_truth_set + confusion_matrix.sum(0) - intersection
    IoU = intersection / (union + 1e-6)
    FWIoU = (ground_truth_set * IoU) / total_pixels
    FWIoU = FWIoU.sum()
    return FWIoU.item()

def compute_dice_coefficient(confusion_matrix):
    intersection = torch.diag(confusion_matrix)
    ground_truth_set = confusion_matrix.sum(1)
    predicted_set = confusion_matrix.sum(0)
    dice = (2 * intersection) / (ground_truth_set + predicted_set + 1e-6)
    mean_dice = torch.mean(dice)
    return mean_dice.item(), dice


2025-02-10 08:21:10.118446: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-10 08:21:10.775790: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [8]:
def load_checkpointed_model_ray(model_name, checkpoint_path, num_classes=None):
    """
    Lädt ein Modell mit Checkpoint ohne Anpassung des Klassifikationslayers.
    """
    # Lade das Modell und den Checkpoint
    loaded_model = TrainedModel(model_name=model_name, width=520, height=520)
    with open(checkpoint_path, "rb") as fp:
        checkpoint_data = pickle.load(fp)

    # Checkpoint in das Modell laden
    loaded_model.model.load_state_dict(checkpoint_data["model_state"], strict=True)

    # (Optional) Optimizer-Zustand laden, falls benötigt
    if "optimizer_state" in checkpoint_data:
        loaded_model.optimizer.load_state_dict(checkpoint_data["optimizer_state"])

    return loaded_model




In [9]:
from Helper.ml_models import MapillaryDataLoader

mapillary_loader = MapillaryDataLoader(
    train_images_dir="/home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/Mapilarry_Vistas/training/images",
    train_annotations_dir="/home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/Mapilarry_Vistas/training_own",
    val_images_dir="/home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/Mapilarry_Vistas/validation/images",
    val_annotations_dir="/home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/Mapilarry_Vistas/validation_own"
)

test_dataset = mapillary_loader.test_dataset

In [10]:
from tqdm import tqdm

def evaluate_model(model: TrainedModel, dataset, num_classes: int) -> dict:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.model.to(device)
    confusion_matrix_total = torch.zeros((num_classes, num_classes), dtype=torch.int64).to(device)
    
    with torch.no_grad():
        for i in tqdm(range(len(dataset)), desc="Evaluating Dataset"):
            image, annotation = dataset[i]
            image = image.to(device)
            annotation = annotation.to(device)
            output = model.inference(image)
            predicted = output.argmax(1).squeeze(0)
            conf_mat = compute_confusion_matrix(predicted.cpu(), annotation.cpu(), num_classes)
            confusion_matrix_total += conf_mat.cpu()
    
    miou, iou_per_class = compute_miou(confusion_matrix_total)
    mpa, pa_per_class = compute_mean_pixel_accuracy(confusion_matrix_total)
    fwiou = compute_fwiou(confusion_matrix_total)
    dice_mean, dice_per_class = compute_dice_coefficient(confusion_matrix_total)
    
    metrics = {
        "mIoU": miou,
        "mPA": mpa,
        "FWIoU": fwiou,
        "Dice_Mean": dice_mean,
        "IoU_per_class": iou_per_class.tolist(),
        "PA_per_class": pa_per_class.tolist(),
        "Dice_per_class": dice_per_class.tolist()
    }
    return metrics


In [11]:
NUM_CLASSES = 124  # Neue Klassenanzahl

for model_name, runs_dict in sorted_hyperparameter_data.items():
    best_run_info = runs_dict["0"]
    checkpoint_path = best_run_info.get("path", None)
    if not checkpoint_path:
        print(f"\n[WARNING] No checkpoint path found for {model_name} run '0'. Skipping.")
        continue
    
    print(f"\nEvaluating model: {model_name}")
    print(f"Checkpoint path:  {checkpoint_path}")

    try:
        model_loaded = load_checkpointed_model_ray(model_name, checkpoint_path, NUM_CLASSES)
    except FileNotFoundError as e:
        print("[WARNING] The exact error was:", e)
        continue

    # Evaluierung auf dem Testset
    metrics = evaluate_model(model_loaded, test_dataset, NUM_CLASSES)
    evaluation_results[model_name] = metrics
    print(f"Results for {model_name}:")
    for k, v in metrics.items():
        if isinstance(v, list):
            print(f"  {k}: [list of length {len(v)}]")
        else:
            print(f"  {k}: {v:.4f}" if isinstance(v, float) else f"  {k}: {v}")

# Optional: Speichere die Ergebnisse
with open("evaluation_results.json", "w") as f:
    json.dump(evaluation_results, f, indent=4)




Evaluating model: deeplabv3_resnet50
Checkpoint path:  /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/HyperparameterLOG/deeplabv3_resnet50/train_hyper_e12c1511_1_auto_cast=True,batch_size=4,learning_rate=0.0001,max_epochs=100,weight_decay=0.0000_2025-02-06_19-30-16/checkpoint_000099/checkpoint.pkl
Using CUDA GPU
Model loaded: deeplabv3_resnet50 | Device: cuda 
Error loading Model with Epoch latest: Error(s) in loading state_dict for DeepLabV3:
	size mismatch for classifier.4.weight: copying a param with shape torch.Size([124, 256, 1, 1]) from checkpoint, the shape in current model is torch.Size([20, 256, 1, 1]).
	size mismatch for classifier.4.bias: copying a param with shape torch.Size([124]) from checkpoint, the shape in current model is torch.Size([20]).
Skipping local .pth load due to error above.


RuntimeError: Error(s) in loading state_dict for DeepLabV3:
	size mismatch for classifier.4.weight: copying a param with shape torch.Size([124, 256, 1, 1]) from checkpoint, the shape in current model is torch.Size([20, 256, 1, 1]).
	size mismatch for classifier.4.bias: copying a param with shape torch.Size([124]) from checkpoint, the shape in current model is torch.Size([20]).

In [ ]:
import json

save_path = "/home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/evaluation_hyperparameter_MapillaryV1.json"

# 2) Ensure the directory exists. If not, create it.
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# 3) Write the evaluation_results dictionary to JSON.
with open(save_path, "w") as f:
    json.dump(evaluation_results, f, indent=4)

print(f"Evaluation results saved to: {save_path}")

Evaluation results saved to: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/evaluation_hyperparameter_cityscapes.json


In [ ]:
best_checkpoints = {}
for model_name, runs_dict in sorted_hyperparameter_data.items():
    # The best run is at index "0"
    best_run_info = runs_dict["0"]
    best_checkpoint_path = best_run_info.get("path", None)
    
    # Store it in our dictionary
    best_checkpoints[model_name] = best_checkpoint_path

# 2) Specify where you want to save the JSON
save_path = "/home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/best_checkpoints.json"

# 3) Ensure the directory exists
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# 4) Write the dictionary to JSON
with open(save_path, "w", encoding="utf-8") as f:
    json.dump(best_checkpoints, f, indent=4)

print(f"Best checkpoint paths saved to: {save_path}")


Best checkpoint paths saved to: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/best_checkpoints.json


In [ ]:
import os
import shutil
import json

# Definiere das Zielverzeichnis
target_dir = "/home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/hyper_city"
os.makedirs(target_dir, exist_ok=True)

# Lade die JSON-Datei mit den besten Checkpoints
best_checkpoints_path = "/home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/best_checkpoints.json"

with open(best_checkpoints_path, "r", encoding="utf-8") as f:
    best_checkpoints = json.load(f)

# Kopiere die besten Checkpoints in das Zielverzeichnis
for model_name, checkpoint_path in best_checkpoints.items():
    if checkpoint_path and os.path.isfile(checkpoint_path):
        # Bestimme den Zielpfad
        dest_checkpoint = os.path.join(target_dir, f"{model_name}_best_checkpoint.pkl")

        # Kopiere die Datei
        shutil.copy2(checkpoint_path, dest_checkpoint)
        print(f"✅ Checkpoint für {model_name} gesichert: {dest_checkpoint}")
    else:
        print(f"⚠️ Kein gültiger Checkpoint für {model_name} gefunden!")

print("✅ Alle Checkpoints wurden gesichert!")


✅ Checkpoint für lraspp_mobilenet_v3_large gesichert: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/hyper_city/lraspp_mobilenet_v3_large_best_checkpoint.pkl
✅ Checkpoint für fcn_resnet101 gesichert: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/hyper_city/fcn_resnet101_best_checkpoint.pkl
✅ Checkpoint für deeplabv3_resnet50 gesichert: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/hyper_city/deeplabv3_resnet50_best_checkpoint.pkl
✅ Checkpoint für fcn_resnet50 gesichert: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/hyper_city/fcn_resnet50_best_checkpoint.pkl
✅ Checkpoint für deeplabv3_mobilenet_v3_large gesichert: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/hyper_city/deeplabv3_mobilenet_v3_large_best_checkpoint.pkl
✅ Checkpoint für deeplabv3_resnet101 gesichert: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_